In [1]:
import importlib

import os
from pathlib import Path


PROJECT_FOLDER = Path(os.getenv("PROJECT_ROOT"))
DATA_FOLDER = PROJECT_FOLDER / "data"
src_path = PROJECT_FOLDER / "src"

import sys
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

In [ ]:
import tortoise.dataset as dataset   # import the module
importlib.reload(dataset)            # reload the module

import tortoise.normalizer as normalizer   # import the module
importlib.reload(normalizer)      # reload the module

import tortoise.dataloader as dataloader   # import the module
importlib.reload(dataloader)            # reload the module


In [ ]:
from tortoise.dataset import *
from tortoise.normalizer import *
from tortoise.dataloader import *

In [9]:
dn = DataNormalizer(DATA_FOLDER / "normalization_stats.json", preloaded=False)

In [12]:
from pathlib import Path
from tortoise.dataloader import build_dataloaders
from tortoise.model import U_Net

DATA_FOLDER = PROJECT_FOLDER / "data"
tiles_dir = DATA_FOLDER / "tiles"

# dn is your previously-computed normalizer
train_loader, val_loader, test_loader = build_dataloaders(
    tiles_dir=tiles_dir,
    batch_size= 128,
    normalizer=dn,
    seed=98,
    train_ratio=0.8,
    val_ratio=0.1,
    save_aug_map_path= DATA_FOLDER / "aug_map.json",
)

Computing masked min/max: 100%|██████████| 15592/15592 [02:32<00:00, 101.97it/s]


In [13]:
len(train_loader.dataset), len(val_loader.dataset), len(test_loader.dataset)

(46776, 5847, 5850)